In [ ]:
from datasets import Datasets

from network import Net

from network_fit import NetworkFit

import savescore
from score import Score
from score_calc import ScoreCalc

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

import numpy as np

In [ ]:
DATASET = 'MNIST'
BATCH_SIZE = 100
NUM_WORKERS = 2

WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.01
steps = 50

SEED = 1

EPOCH = 100

In [ ]:
torch.cuda.manual_seed_all(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

In [ ]:
codename = 'cnn'

total_loss_name = codename + "_total_loss"
acc_name = codename + "_accuracy"

data_text_name = codename + "_data"

In [ ]:
trans = transform=transforms.Compose([transforms.ToTensor()])
instance_datasets = Datasets(DATASET, BATCH_SIZE, NUM_WORKERS, transform = trans)
data_sets = instance_datasets.create()

trainloader = data_sets[0]
testloader = data_sets[1]
classes = data_sets[2]
based_labels = data_sets[3]
trainset = data_sets[4]
testset = data_sets[5]

In [ ]:
model = Net().cuda()

optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=steps, gamma=0.1)

soft_criterion = nn.CrossEntropyLoss()

fit = NetworkFit(model, optimizer, soft_criterion)

In [ ]:
loss = Score()
correct = Score()
score_loss = [loss]
score_correct = [correct]
sc = ScoreCalc(score_loss, score_correct, BATCH_SIZE)

In [ ]:
for epoch in range(EPOCH):
    print('epoch', epoch+1)

    for (inputs, labels) in trainloader:
        inputs, labels = inputs.cuda(), labels.cuda()
        
        fit.train(inputs, labels)
    
    for (inputs, labels) in trainloader:
        inputs, labels = inputs.cuda(), labels.cuda()
        
        losses, corrects = fit.test(inputs, labels)
        
        sc.calc_sum(losses, corrects)
    
    sc.score_print(len(trainset))
    sc.score_append(len(trainset))
    sc.score_del()
    
    
    for (inputs, labels) in testloader:
        inputs, labels = inputs.cuda(), labels.cuda()
        
        losses, corrects = fit.test(inputs, labels)
        
        sc.calc_sum(losses, corrects)
    
    sc.score_print(len(testset), train = False)
    sc.score_append(len(testset), train = False)
    sc.score_del()
    
    scheduler.step()

In [ ]:
train_losses, train_corrects = sc.get_value()
test_losses, test_corrects = sc.get_value(train = False)
savescore.save_data(train_losses[0], test_losses[0], train_corrects[0], test_corrects[0], data_text_name)

In [ ]:
savescore.plot_score(EPOCH, train_losses[0], test_losses[0], y_lim = 2.0, y_label = 'LOSS', legend = ['train loss', 'test loss'], title = 'total loss', filename = total_loss_name)
savescore.plot_score(EPOCH, train_corrects[0], test_corrects[0], y_lim = 1, y_label = 'ACCURACY', legend = ['train acc', 'test acc'], title = 'accuracy', filename = acc_name)

In [ ]:
torch.save(model.state_dict(), 'model.pth')